In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

import numpy as np
from random import randint
from math import floor, ceil
import codecs

In [ ]:
from bulletin import root, default_input, default_output, agora, hoje, ontem, anteontem, dias_apos, dias_apos_label
from bulletin.services.metabase import Metabase
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica
from bulletin.utils.static import Municipios
from bulletin.utils import utils, static
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import trim_overspace
from bulletin.utils.normalize import normalize_text#normalize_hash, normalize_labels, , date_hash, normalize_number

In [ ]:
from datetime import datetime, date, timedelta

exclusao_pathfile = join(root, 'database', 'casos_confirmados')

today = pd.to_datetime(date.today())
ontem = today - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

In [ ]:
municipios = Municipios()
municipios['mun_resid'] = municipios['municipio']
municipios.loc[municipios['uf']!='PR','mun_resid'] = municipios.loc[municipios['uf']!='PR','municipio'] + '/' + municipios['uf']

In [ ]:
cc = CasosConfirmados()
try:
    cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
    print(f"cc_{hoje.strftime('%d_%m_%Y')}.pkl loaded")
except:
    cc.load(f"cc_{ontem.strftime('%d_%m_%Y')}", compress=True)
    print(f"cc_{ontem.strftime('%d_%m_%Y')}.pkl loaded")

In [ ]:
used_columns = ['id_notifica', 'paciente', 'sexo', 'nome_mae', 'idade', 'cpf', 'cns', 'data_diagnostico', 'evolucao', 'data_cura_obito', 'uf_residencia','classificacao_final', 'numero_do', 'ibge_residencia']
cc.df = cc.df.drop(columns=[col for col in cc.df.columns if not col in used_columns])

cc.df = pd.merge(cc.df.rename(columns={'ibge_residencia':'ibge'}),municipios[['ibge','rs','mun_resid','uf','municipio']],on='ibge',how='left').rename(columns={'ibge':'ibge_resid'})

In [ ]:
cc.df.columns

In [ ]:
df_casos = cc.df[['id_notifica', 'paciente', 'sexo', 'nome_mae', 'idade', 'cpf', 'cns', 'data_diagnostico', 'evolucao', 'data_cura_obito', 'uf_residencia', 'municipio', 'rs','classificacao_final', 'numero_do']].copy()
df_casos['nome_mae'] = df_casos['nome_mae'].fillna('NÃO CONSTA')

In [ ]:
id_notifica = 3000265
name='GENILSON FRANCISCO'
nome_mae='ANESTINA'
rs = 10

In [ ]:
##PACIENTE
df_casos.loc[df_casos['paciente'].str.contains(str.upper(name))]

In [ ]:
##PACIENTE E NOME MAE
df_casos.loc[(df_casos['paciente'].str.contains(str.upper(name))) & (df_casos['nome_mae'].str.contains(str.upper(nome_mae)))]

In [ ]:
##PACIENTE E RS
df_casos.loc[(df_casos['paciente'].str.contains(str.upper(name))) & (df_casos['rs'] == rs)]

In [ ]:
##ID NOTIFICA
df_casos.loc[df_casos['id_notifica'] == id_notifica]

In [ ]:
##NOME MAE
df_casos.loc[(df_casos['nome_mae'].str.contains(str.upper(nome_mae)))]

In [ ]:
df_casos.loc[df_casos['paciente'] == (str.upper(name))]

In [ ]:
# list_verification = [5402543,2546746,1604429,3068694,2755686, 2890203]
# df_casos.loc[df_casos['id_notifica'].isin(list_verification)]